In [2]:
import warnings
from gensim import matutils
warnings.filterwarnings('ignore')
from gensim.models import KeyedVectors
from urllib import request
import string
from nltk.corpus import brown
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.stem.porter import *
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from gensim.utils import simple_preprocess
from gensim import corpora
import re
from nltk.stem.porter import PorterStemmer  
from nltk import pos_tag
porter_stemmer = PorterStemmer() 
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('brown')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')
punc = '[,.!\')(-;?:''"``""]'
from nltk import sent_tokenize, word_tokenize
import pandas as pd

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import json, pickle
def load_dict(file):
    f = open(file, 'r')
    js =f.read()
    DIC = json.loads(js)
    f.close()
    return DIC

# 获取字典信息
f1 = open('corpus/GlossDict.txt', 'r') 
f2 = open('corpus/GlossDictExpand.txt', 'r')
dic = eval(f1.read())   # str into dict
dic2 = eval(f2.read())
glossDict = dic
ExpandglossDict = dic2
f1.close()
f2.close()

# 获取文章信息和文章ID
f3 = open('corpus/ID.txt', 'r')
f4 = open('corpus/INSTANCE.txt', 'r')
id_list = [] # 3944
article = [] # 3944
for line in f3.readlines():
    line = line.strip('\n')
    line = line.strip('"')
    id_list.append(line)

for art in f4.readlines():
    art = art.strip('\n')
    art = art.strip('[,],\\n,",')
    article.append(art)
f3.close()
f4.close()
f6 = open('corpus/GlossDictWITHpunc.txt', 'r') 
dic6 = eval(f6.read())   # str into dict
glossWithPunc = list(dic6)
f6.close()
EPGwithSTOP = open('corpus/EPGwithSTOP.txt', 'r') 
dicEPG = eval(EPGwithSTOP.read())  
EPGwith = list(dicEPG)                                 # wrod 保留stopwords； 额外信息没有stopwords
EPGwithSTOP.close()
# ----------corpus------------
with open(r"corpus/corp.txt", "r") as fo:
    out = fo.read()
    sentence_corpus = json.loads(out)

#with open(r'new-corpus/new-corpus-list-news-2010-1M.txt', 'rb') as fp:
#    sentence_corpus2 = pickle.load(fp)

#with open(r'new-corpus/new-corpus-wiki-2016-1M.txt', 'rb') as fp:
#    sentence_corpus3 = pickle.load(fp)
# dict
glossAND_stop = load_dict('./corpus/glossAND_stop_dict.txt') # 包含stopwords的词典信息
glossDict = dic                                              # 不包含stopwrods的词典信息
hypANDstop = load_dict('./corpus/hypANDstop_dict.txt')    # 加入额外信息的包含stopwords的词典信息
hypAND = load_dict('./corpus/hypAND_dict.txt')            # 加入额外信息的不包含stopwords的词典信息
EPGwith = list(dicEPG)        
print(len(sentence_corpus))
print('ok')

193228
ok


In [4]:
f_dict_map = open('./corpus/EnglishLS.dictionary.mapping.xml', 'r')
soup1= BeautifulSoup(f_dict_map) 
re_synset = re.compile(r'[a-z]+\.[a-z]\.\d{2}')
noun_key_value = re.compile(r'[a-z]+[%][\d]+[:][\d]+[:][\d]+[:]+[a-z]*[:]*[\d\d]*')

list_gloss_t = []  
#dict_gloss = {}.fromkeys(('id','gloss'))
for ul in soup1.find_all(name='lexelt'):
    item = ul.attrs['item']
    
    for se in ul.find_all(name='sense'):
        id = se.attrs['id']
        glo = se.attrs['gloss']
        glo = re.sub(punc,'',glo)
        token_glo = nltk.word_tokenize(glo)
        token_glo.append(item)
        #token_glo = [word for word in token_glo if word not in stop_words]  #去stopwords
        #token_glo = [word for word in token_glo if word not in punc]     #去punc
        #token_glo = [word for word in token_glo if word in word_vectors.wv.vocab] #.vocab
        #dict_gloss['id']=id
        #dict_gloss['gloss']=token_glo
        list_gloss_t.append((id, token_glo)) 
        

In [5]:
DIC = {id: gloss for (id, gloss) in list_gloss_t}  

In [18]:
from pandas import Series,DataFrame
import pandas as pd
import openpyxl
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [13]:
def gen_merge_df(L):
    lTarget = []
    lBefore = []
    lGloss = []
    lAfter = []
    n = 0
    length = len(L)
    while n < length:
        id = L[n][0]
        new_id = L[n][1]
        sub = []
        next_n = n
        while new_id == L[next_n][1]:
            sub_id = L[next_n][0]
            sub_new = L[next_n][1]
            sub.append((sub_id, sub_new))
            next_n += 1
            if next_n > length-1:

                break
        n = next_n
        for item in sub:
            Target = DIC[item[0]][-1]
            Before = item[0]
            Gloss = ' '.join(DIC[item[0]][:-1])
            After = item[1]
            lTarget.append(Target)
            lBefore.append(Before)
            lGloss.append(Gloss)
            lAfter.append(After)

    noun  = {'Target word':lTarget,
           'Before merge':lBefore,
           'Gloss':lGloss,
           'After merge':lAfter}

    merge_df = pd.DataFrame(noun)
    
    return merge_df

In [20]:
verb_df = gen_merge_df(merge_targt_adj)
#noun_df = noun_df.set_index('Target word')
#noun_df.to_excel('merge_noun.xlsx')
verb_df

,Target word,Before merge,Gloss,After merge
0,hot.a,hot%3:00:01::,used of physical heat having a high or higher ...,a
1,hot.a,hot%3:00:02::,extended meanings especially of psychological ...,a
2,hot.a,hot%5:00:00:active:01,informal marked by excited activity,b
3,hot.a,hot%5:00:00:charged:00,electricity charged or energized with electricity,b
4,hot.a,hot%5:00:00:fast:01,very fast,b
5,hot.a,hot%5:00:00:fresh:01,newly made,b
6,hot.a,hot%5:00:00:good:01,very good often used in the negative,b
7,hot.a,hot%5:00:00:illegal:00,informal recently stolen or smuggled,b
8,hot.a,hot%5:00:00:lucky:00,having or bringing unusually good luck,b
9,hot.a,hot%5:00:00:near:00,of a seeker near to the object sought,b


In [10]:
LS_v = [(0,114,'activate',0,5),(114,246,'add',5,11),(246,379,'appear',11,14),(623,754,'ask',25,31),
     
        (1067,1146,'begin',51,55),(1146,1213,'climb',55,60),(1690,1777,'eat',89,96),
      
        (1777,1842,'encounter',96,100),
      
        (1975,2007,'hear',107,114),(2268,2304,'lose',162,171),(2304,2344,'mean',171,178),(2344,2374,'miss',178,186)
     
        ,(2919,2971,'play',221,233),(2971,3065,'produce',233,239),(3065,3134,'provide',239,245),
     
        (3134,3161,'receive',245,254),(3161,3231,'remain',254,257),(3231,3261,'rule',257,262),(3377,3432,'smell',273,280),
     
        (3589,3653,'suspend',305,312),(3653,3726,'talk',312,321),(3726,3783,'treat',321,330),(3783,3797,'use',330,335),
     
        (3797,3831,'wash',335,347),(3831,3882,'watch',347,354),(3882,3921,'win',354,361),(3921,3944,'write',361,369)]
      

LS_n = [(379,490,'argument',14,19),(490,623,'arm',19,25),
         
         (935,1067,'bank',41,51),(1275,1403,'degree',64,71),(1403,1517,'difference',71,76),(1567,1590,'difficulty',81,85),
         
         (1590,1690,'disc',85,89),(2050,2124,'image',136,143),(2143,2236,'interest',148,155),(2236,2268,'judgment',155,162),
         (2515,2632,'paper',201,208),(2632,2748,'party',208,213),(2748,2835,'performance',213,218),
         
         (2835,2919,'plan',218,221),(3261,3359,'shelter',262,266),(3461,3557,'sort',294,298),(3557,3589,'source',298,305)]
 

LS_adj = [(1517,1567,'different',76,81),(2007,2050,'hot',114,136),
          (2124,2143,'important',143,148),(3432,3461,'solid',280,294)]

In [9]:
merge_targt_noun = [
('bank%1:06:00::','b'),
('bank%1:06:01::','b'),
('bank%1:14:00::','b'),
('bank%1:14:01::','b'),

('bank%1:17:00::','c'),
('bank%1:17:01::','c'),
('bank%1:17:02::','c'),

('bank%1:21:00::','d'),
('bank%1:21:01::','d'),

('degree%1:07:00::','e'),
('degree%1:07:01::','e'),
('degree%1:26:01::','e'),

('image%1:06:00::','f'),
('image%1:06:01::','f'),

('image%1:07:00::','g'),
('image%1:09:00::','g'),
('image%1:09:02::','g'),

('interest%1:04:01::','h'),
('interest%1:07:01::','h'),
('interest%1:07:02::','h'),
('interest%1:09:00::','h'),
('interest%1:14:00::','h'),
('interest%1:21:00::','i'),
('interest%1:21:03::','i'),


('judgment%1:04:00::','j'),
('judgment%1:04:02::','j'),
('judgment%1:10:00::','j'),
('judgment%1:07:00::','k'),
('judgment%1:09:00::','k'),
('judgment%1:09:01::','k'),
('judgment%1:09:04::','k'),


('paper%1:10:01::','l'),
('paper%1:10:02::','l'),
('paper%1:06:00::','m'),
('paper%1:10:03::','m'),
('paper%1:14:00::','m'),

('paper%1:10:00::','n'),
('paper%1:27:00::','n'),

('party%1:11:00::','o'),
('party%1:14:02::','o'),
('party%1:18:00::','o'),

('performance%1:04:00::','p'),
('performance%1:04:01::','p'),
('performance%1:10:00::','p'),

('performance%1:04:03::','q'),
('performance%1:22:00::','q'),

('plan%1:09:00::','r'),
('plan%1:09:01::','r'),

('shelter%1:06:00::','s'),
('shelter%1:06:01::','s'),

('shelter%1:21:00::','t'),
('shelter%1:26:00::','t'),

('source%1:10:00::','u'),
('source%1:10:01::','u'),
('source%1:18:00::','u'),

]

merge_targt_verb = [

('42602','a'),
('42605','a'),
('238101','b'),
('238105','b'),
('238102','c'),
('238104','c'),
('238106','c'),  
('770001','d'),
('770002','d'),  
('770005','d'),
('1297001','e'),
('1297006','e'),
('1297002','f'),
('1297007','f'),
('1353101','g'), 
('1353103','g'),
('1892101','h'),
('1892103','h'),
('1892105','h'),

('1892102','i'),
('1892104','i'),
('1892106','i'),
('1892107','i'),

('2439901','j'),
('2439902','j'),
('2439904','j'),
('2439905','j'),

('2439903','k'),
('2439908','k'),
('2439906','k'),

('2555501','l'),
('2555502','l'),
('2555507','l'),

('2555504','m'),
('2555505','m'),
('2555506','m'),

('2644301','n'),
('2644307','n'),
('2644302','n'),
('2644303','n'),  
('3165218','o'),
('3165217','o'),
('3165213','o'),
('3165220','p'),
('3165214','p'),
('3288301','q'),
('3288306','q'),
('3288302','r'),
('3288305','r'),

('3313901','s'),
('3313906','s'),
('3313902','t'),
('3313905','t'),

('3434801','u'),
('3434806','u'),

('3434803','v'),
('3434807','v'),

('3477801','w'),
('3477803','w'),

('3597906','x'),
('3597907','x'),
('3597911','x'),

('3893501','y'),
('3893505','y'),
('3893507','y'),
('3893508','y'),

('3893502','z'),
('3893503','z'),
('3893509','z'),

('4155302','aa'),
('4155303','aa'),
('4155304','aa'),
('4155307','aa'),

('4198501','bb'),
('4198502','bb'),
('4198503','bb'),
('4198506','bb'),
('4198507','bb'),  

('4380101','cc'),
('4380102','cc'),
('4380103','cc'),

('4380104','dd'),
('4380105','dd'),
('4380106','ee'),
('4380109','ee'),
('4530702','ff'),
('4530704','ff'),
('4636102','gg'), 
('4636101','gg'), 
('4636107','gg'), 
('4636108','gg'), 
('4636103','hh'), 
('4636104','hh'), 
('4636110','hh'), 
('4636111','hh'), 
('4636112','hh'), 
('4640501','ii'), 
('4640502','ii'), 
('4640507','ii'),
('4640503','jj'), 
('4640508','jj'), 
('4640509','jj'), 
('4711401','kk'), 
('4711403','kk'), 
('4711405','kk'),
('4711404','ll'), 
('4711406','ll'), 
('4753401','mm'), 
('4753404','mm'), 
('4753406','mm'),
('4753403','nn'), 
('4753407','nn'),
('4753405','oo'), 
('4753408','oo'),
]

merge_targt_adj = [
('hot%3:00:01::','a'),
('hot%3:00:02::','a'),
('hot%5:00:00:active:01','b'),
('hot%5:00:00:charged:00','b'),
('hot%5:00:00:fast:01','b'),
('hot%5:00:00:fresh:01','b'),
('hot%5:00:00:good:01','b'),
('hot%5:00:00:illegal:00','b'),
('hot%5:00:00:lucky:00','b'),
('hot%5:00:00:near:00','b'),
('hot%5:00:00:new:00','b'),
('hot%5:00:00:popular:00','b'),
('hot%5:00:00:pungent:00','b'),
('hot%5:00:00:radioactive:00','b'),
('hot%5:00:00:sexy:00','b'),
('hot%5:00:00:skilled:00','b'),
('hot%5:00:00:unpleasant:00','b'),
('hot%5:00:00:violent:00','b'),
('hot%5:00:00:wanted:00','b'),
('hot%5:00:00:warm:03','b'),
('hot%5:00:02:fast:01','b'),
('important%3:00:00::','c'),
('important%3:00:02::','c'),
('important%3:00:04::','c'),
('important%5:00:00:immodest:02','d'),
('important%5:00:00:influential:00','d'),
('solid%3:00:01::','e'),
('solid%3:00:02::','e'),
('solid%5:00:00:homogeneous:00','f'),
('solid%5:00:00:honorable:00','f'),
('solid%5:00:00:opaque:00','f'),
('solid%5:00:00:plain:02','f'),
('solid%5:00:00:sound:01','f'),
('solid%5:00:00:unbroken:02','f'),
('solid%5:00:00:undiversified:00','f'),
('solid%5:00:00:wholesome:00','f'),
('solid%5:00:00:hard:01','f'),
('solid%5:00:00:good:01','f'),
('solid%5:00:00:frozen:00','f'),
('solid%5:00:00:cubic:00','f'),
]

In [6]:
#Noune  合并后平均意思3.58， 17个单词， 1570个例子
def sense_merge():
    f_noun_answer.replace('argument%1:10:00::','a',inplace=True)
    f_noun_answer.replace('argument%1:10:00::','a',inplace=True)
    f_noun_answer.replace('bank%1:06:00::','b',inplace=True)
    f_noun_answer.replace('bank%1:06:01::','b',inplace=True)
    f_noun_answer.replace('bank%1:14:00::','b',inplace=True)
    f_noun_answer.replace('bank%1:14:01::','b',inplace=True)

    f_noun_answer.replace('bank%1:17:00::','c',inplace=True)
    f_noun_answer.replace('bank%1:17:01::','c',inplace=True)
    f_noun_answer.replace('bank%1:17:02::','c',inplace=True)

    f_noun_answer.replace('bank%1:21:00::','d',inplace=True)
    f_noun_answer.replace('bank%1:21:01::','d',inplace=True)

    f_noun_answer.replace('degree%1:07:00::','e',inplace=True)
    f_noun_answer.replace('degree%1:07:01::','e',inplace=True)
    f_noun_answer.replace('degree%1:26:01::','e',inplace=True)

    f_noun_answer.replace('image%1:06:00::','f',inplace=True)
    f_noun_answer.replace('image%1:06:01::','f',inplace=True)

    f_noun_answer.replace('image%1:07:00::','g',inplace=True)
    f_noun_answer.replace('image%1:09:00::','g',inplace=True)
    f_noun_answer.replace('image%1:09:02::','g',inplace=True)

    f_noun_answer.replace('interest%1:04:01::','h',inplace=True)
    f_noun_answer.replace('interest%1:07:01::','h',inplace=True)
    f_noun_answer.replace('interest%1:07:02::','h',inplace=True)
    f_noun_answer.replace('interest%1:09:00::','h',inplace=True)
    f_noun_answer.replace('interest%1:14:00::','h',inplace=True)
    f_noun_answer.replace('interest%1:21:00::','i',inplace=True)
    f_noun_answer.replace('interest%1:21:03::','i',inplace=True)

    #------------------
    f_noun_answer.replace('judgment%1:04:00::','j',inplace=True)
    f_noun_answer.replace('judgment%1:04:02::','j',inplace=True)
    f_noun_answer.replace('judgment%1:10:00::','j',inplace=True)
    f_noun_answer.replace('judgment%1:07:00::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:00::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:01::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:04::','k',inplace=True)
    #----------------

    f_noun_answer.replace('paper%1:10:01::','l',inplace=True)
    f_noun_answer.replace('paper%1:10:02::','l',inplace=True)
    f_noun_answer.replace('paper%1:06:00::','m',inplace=True)
    f_noun_answer.replace('paper%1:10:03::','m',inplace=True)
    f_noun_answer.replace('paper%1:14:00::','m',inplace=True)

    f_noun_answer.replace('paper%1:10:00::','n',inplace=True)
    f_noun_answer.replace('paper%1:27:00::','n',inplace=True)

    f_noun_answer.replace('party%1:11:00::','o',inplace=True)
    f_noun_answer.replace('party%1:14:02::','o',inplace=True)
    f_noun_answer.replace('party%1:18:00::','o',inplace=True)

    f_noun_answer.replace('performance%1:04:00::','p',inplace=True)
    f_noun_answer.replace('performance%1:04:01::','p',inplace=True)
    f_noun_answer.replace('performance%1:10:00::','p',inplace=True)

    f_noun_answer.replace('performance%1:04:03::','q',inplace=True)
    f_noun_answer.replace('performance%1:22:00::','q',inplace=True)

    f_noun_answer.replace('plan%1:09:00::','r',inplace=True)
    f_noun_answer.replace('plan%1:09:01::','r',inplace=True)

    f_noun_answer.replace('shelter%1:06:00::','s',inplace=True)
    f_noun_answer.replace('shelter%1:06:01::','s',inplace=True)

    f_noun_answer.replace('shelter%1:21:00::','t',inplace=True)
    f_noun_answer.replace('shelter%1:26:00::','t',inplace=True)

    f_noun_answer.replace('source%1:10:00::','u',inplace=True)
    f_noun_answer.replace('source%1:10:01::','u',inplace=True)
    f_noun_answer.replace('source%1:18:00::','u',inplace=True)


    #
    #answer
    #=-----------------
    f_noun_true.replace('argument%1:10:00::','a',inplace=True)
    f_noun_true.replace('argument%1:10:00::','a',inplace=True)

    f_noun_true.replace('bank%1:06:00::','b',inplace=True)
    f_noun_true.replace('bank%1:06:01::','b',inplace=True)
    f_noun_true.replace('bank%1:14:00::','b',inplace=True)
    f_noun_true.replace('bank%1:14:01::','b',inplace=True)

    f_noun_true.replace('bank%1:17:00::','c',inplace=True)
    f_noun_true.replace('bank%1:17:01::','c',inplace=True)
    f_noun_true.replace('bank%1:17:02::','c',inplace=True)

    f_noun_true.replace('bank%1:21:00::','d',inplace=True)
    f_noun_true.replace('bank%1:21:01::','d',inplace=True)

    f_noun_true.replace('degree%1:07:00::','e',inplace=True)
    f_noun_true.replace('degree%1:07:01::','e',inplace=True)
    f_noun_true.replace('degree%1:26:01::','e',inplace=True)

    f_noun_true.replace('image%1:06:00::','f',inplace=True)
    f_noun_true.replace('image%1:06:01::','f',inplace=True)

    f_noun_true.replace('image%1:07:00::','g',inplace=True)
    f_noun_true.replace('image%1:09:00::','g',inplace=True)
    f_noun_true.replace('image%1:09:02::','g',inplace=True)

    f_noun_true.replace('interest%1:04:01::','h',inplace=True)
    f_noun_true.replace('interest%1:07:01::','h',inplace=True)
    f_noun_true.replace('interest%1:07:02::','h',inplace=True)
    f_noun_true.replace('interest%1:09:00::','h',inplace=True)
    f_noun_true.replace('interest%1:14:00::','h',inplace=True)
    f_noun_true.replace('interest%1:21:00::','i',inplace=True)
    f_noun_true.replace('interest%1:21:03::','i',inplace=True)

    #------------------
    f_noun_true.replace('judgment%1:04:00::','j',inplace=True)
    f_noun_true.replace('judgment%1:04:02::','j',inplace=True)
    f_noun_true.replace('judgment%1:10:00::','j',inplace=True)
    f_noun_true.replace('judgment%1:07:00::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:00::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:01::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:04::','k',inplace=True)
    #----------------

    f_noun_true.replace('paper%1:10:01::','l',inplace=True)
    f_noun_true.replace('paper%1:10:02::','l',inplace=True)
    f_noun_true.replace('paper%1:06:00::','m',inplace=True)
    f_noun_true.replace('paper%1:10:03::','m',inplace=True)
    f_noun_true.replace('paper%1:14:00::','m',inplace=True)

    f_noun_true.replace('paper%1:10:00::','n',inplace=True)
    f_noun_true.replace('paper%1:27:00::','n',inplace=True)

    f_noun_true.replace('party%1:11:00::','o',inplace=True)
    f_noun_true.replace('party%1:14:02::','o',inplace=True)
    f_noun_true.replace('party%1:18:00::','o',inplace=True)

    f_noun_true.replace('performance%1:04:00::','p',inplace=True)
    f_noun_true.replace('performance%1:04:01::','p',inplace=True)
    f_noun_true.replace('performance%1:10:00::','p',inplace=True)

    f_noun_true.replace('performance%1:04:03::','q',inplace=True)
    f_noun_true.replace('performance%1:22:00::','q',inplace=True)

    f_noun_true.replace('plan%1:09:00::','r',inplace=True)
    f_noun_true.replace('plan%1:09:01::','r',inplace=True)

    f_noun_true.replace('shelter%1:06:00::','s',inplace=True)
    f_noun_true.replace('shelter%1:06:01::','s',inplace=True)

    f_noun_true.replace('shelter%1:21:00::','t',inplace=True)
    f_noun_true.replace('shelter%1:26:00::','t',inplace=True)

    f_noun_true.replace('source%1:10:00::','u',inplace=True)
    f_noun_true.replace('source%1:10:01::','u',inplace=True)
    f_noun_true.replace('source%1:18:00::','u',inplace=True)

    #Verb  合并后意思 4.07， 27个单词， 1698 个例子

    f_answer.replace('42602','a',inplace=True)
    f_answer.replace('42605','a',inplace=True)
    f_answer.replace('238101','b',inplace=True)
    f_answer.replace('238105','b',inplace=True)
    f_answer.replace('238102','c',inplace=True)
    f_answer.replace('238104','c',inplace=True)
    f_answer.replace('238106','c',inplace=True)  # delete 'begin'
    f_answer.replace('770001','d',inplace=True)
    f_answer.replace('770002','d',inplace=True)   #delete 'decide'
    f_answer.replace('770005','d',inplace=True)
    f_answer.replace('1297001','e',inplace=True)
    f_answer.replace('1297006','e',inplace=True)
    f_answer.replace('1297002','f',inplace=True)
    f_answer.replace('1297007','f',inplace=True)
    f_answer.replace('1353101','g',inplace=True)  #delete expect,express
    f_answer.replace('1353103','g',inplace=True)
    f_answer.replace('1892101','h',inplace=True)
    f_answer.replace('1892103','h',inplace=True)
    f_answer.replace('1892105','h',inplace=True)

    f_answer.replace('1892102','i',inplace=True)
    f_answer.replace('1892104','i',inplace=True)
    f_answer.replace('1892106','i',inplace=True)
    f_answer.replace('1892107','i',inplace=True)

    f_answer.replace('2439901','j',inplace=True)
    f_answer.replace('2439902','j',inplace=True)
    f_answer.replace('2439904','j',inplace=True)
    f_answer.replace('2439905','j',inplace=True)

    f_answer.replace('2439903','k',inplace=True)
    f_answer.replace('2439908','k',inplace=True)
    f_answer.replace('2439906','k',inplace=True)

    f_answer.replace('2555501','l',inplace=True)
    f_answer.replace('2555502','l',inplace=True)
    f_answer.replace('2555507','l',inplace=True)

    f_answer.replace('2555504','m',inplace=True)
    f_answer.replace('2555505','m',inplace=True)
    f_answer.replace('2555506','m',inplace=True)

    f_answer.replace('2644301','n',inplace=True)
    f_answer.replace('2644307','n',inplace=True)
    f_answer.replace('2644302','n',inplace=True)
    f_answer.replace('2644303','n',inplace=True)   # delete note

    f_answer.replace('3165218','o',inplace=True)
    f_answer.replace('3165217','o',inplace=True)
    f_answer.replace('3165213','o',inplace=True)

    f_answer.replace('3165220','p',inplace=True)
    f_answer.replace('3165214','p',inplace=True)

    f_answer.replace('3288301','q',inplace=True)
    f_answer.replace('3288306','q',inplace=True)

    f_answer.replace('3288302','r',inplace=True)
    f_answer.replace('3288305','r',inplace=True)

    f_answer.replace('3313901','s',inplace=True)
    f_answer.replace('3313906','s',inplace=True)
    f_answer.replace('3313902','t',inplace=True)
    f_answer.replace('3313905','t',inplace=True)

    f_answer.replace('3434801','u',inplace=True)
    f_answer.replace('3434806','u',inplace=True)

    f_answer.replace('3434803','v',inplace=True)
    f_answer.replace('3434807','v',inplace=True)

    f_answer.replace('3477801','w',inplace=True)
    f_answer.replace('3477803','w',inplace=True)

    f_answer.replace('3597906','x',inplace=True)
    f_answer.replace('3597907','x',inplace=True)
    f_answer.replace('3597911','x',inplace=True)

    f_answer.replace('3893501','y',inplace=True)
    f_answer.replace('3893505','y',inplace=True)
    f_answer.replace('3893507','y',inplace=True)
    f_answer.replace('3893508','y',inplace=True)

    f_answer.replace('3893502','z',inplace=True)
    f_answer.replace('3893503','z',inplace=True)
    f_answer.replace('3893509','z',inplace=True)

    f_answer.replace('4155302','aa',inplace=True)
    f_answer.replace('4155303','aa',inplace=True)
    f_answer.replace('4155304','aa',inplace=True)
    f_answer.replace('4155307','aa',inplace=True)

    f_answer.replace('4198501','bb',inplace=True)
    f_answer.replace('4198502','bb',inplace=True)
    f_answer.replace('4198503','bb',inplace=True)
    f_answer.replace('4198506','bb',inplace=True)
    f_answer.replace('4198507','bb',inplace=True)  

    f_answer.replace('4380101','cc',inplace=True)
    f_answer.replace('4380102','cc',inplace=True)
    f_answer.replace('4380103','cc',inplace=True)

    f_answer.replace('4380104','dd',inplace=True)
    f_answer.replace('4380105','dd',inplace=True)

    #--------------------------------------------
    f_answer.replace('4380106','ee',inplace=True)
    f_answer.replace('4380109','ee',inplace=True)

    f_answer.replace('4530702','ff',inplace=True)
    f_answer.replace('4530704','ff',inplace=True)

    f_answer.replace('4636102','gg',inplace=True) 
    f_answer.replace('4636101','gg',inplace=True) 
    f_answer.replace('4636107','gg',inplace=True) 
    f_answer.replace('4636108','gg',inplace=True) 

    f_answer.replace('4636103','hh',inplace=True) 
    f_answer.replace('4636104','hh',inplace=True) 
    f_answer.replace('4636110','hh',inplace=True) 
    f_answer.replace('4636111','hh',inplace=True) 
    f_answer.replace('4636112','hh',inplace=True) 
    #---------------


    f_answer.replace('4640501','ii',inplace=True) 
    f_answer.replace('4640502','ii',inplace=True) 
    f_answer.replace('4640507','ii',inplace=True)

    f_answer.replace('4640503','jj',inplace=True) 
    f_answer.replace('4640508','jj',inplace=True) 
    f_answer.replace('4640509','jj',inplace=True) 
    #------------------
    f_answer.replace('4711401','kk',inplace=True) 
    f_answer.replace('4711403','kk',inplace=True) 
    f_answer.replace('4711405','kk',inplace=True)

    f_answer.replace('4711404','ll',inplace=True) 
    f_answer.replace('4711406','ll',inplace=True) 

    #---------------------------------
    f_answer.replace('4753401','mm',inplace=True) 
    f_answer.replace('4753404','mm',inplace=True) 
    f_answer.replace('4753406','mm',inplace=True)

    f_answer.replace('4753403','nn',inplace=True) 
    f_answer.replace('4753407','nn',inplace=True)

    f_answer.replace('4753405','oo',inplace=True) 
    f_answer.replace('4753408','oo',inplace=True)

    #
    # 答案
    #---------------------------------------------------------
    f_true.replace('42602','a',inplace=True)
    f_true.replace('42605','a',inplace=True)
    f_true.replace('238101','b',inplace=True)
    f_true.replace('238105','b',inplace=True)
    f_true.replace('238102','c',inplace=True)
    f_true.replace('238104','c',inplace=True)
    f_true.replace('238106','c',inplace=True)  # delete 'begin'
    f_true.replace('770001','d',inplace=True)
    f_true.replace('770002','d',inplace=True)   #delete 'decide'
    f_true.replace('770005','d',inplace=True)
    f_true.replace('1297001','e',inplace=True)
    f_true.replace('1297006','e',inplace=True)
    f_true.replace('1297002','f',inplace=True)
    f_true.replace('1297007','f',inplace=True)
    f_true.replace('1353101','g',inplace=True)  #delete expect,express
    f_true.replace('1353103','g',inplace=True)
    f_true.replace('1892101','h',inplace=True)
    f_true.replace('1892103','h',inplace=True)
    f_true.replace('1892105','h',inplace=True)
    f_true.replace('1892102','i',inplace=True)
    f_true.replace('1892104','i',inplace=True)
    f_true.replace('1892106','i',inplace=True)
    f_true.replace('1892107','i',inplace=True)
    f_true.replace('2439901','j',inplace=True)
    f_true.replace('2439902','j',inplace=True)
    f_true.replace('2439904','j',inplace=True)
    f_true.replace('2439905','j',inplace=True)
    f_true.replace('2439903','k',inplace=True)
    f_true.replace('2439908','k',inplace=True)
    f_true.replace('2439906','k',inplace=True)
    f_true.replace('2555501','l',inplace=True)
    f_true.replace('2555502','l',inplace=True)
    f_true.replace('2555507','l',inplace=True)
    f_true.replace('2555504','m',inplace=True)
    f_true.replace('2555505','m',inplace=True)
    f_true.replace('2555506','m',inplace=True)
    f_true.replace('2644301','n',inplace=True)
    f_true.replace('2644307','n',inplace=True)
    f_true.replace('2644302','n',inplace=True)
    f_true.replace('2644303','n',inplace=True)   # delete note
    f_true.replace('3165218','o',inplace=True)
    f_true.replace('3165217','o',inplace=True)
    f_true.replace('3165213','o',inplace=True)
    f_true.replace('3165220','p',inplace=True)
    f_true.replace('3165214','p',inplace=True)
    f_true.replace('3288301','q',inplace=True)
    f_true.replace('3288306','q',inplace=True)
    f_true.replace('3288302','r',inplace=True)
    f_true.replace('3288305','r',inplace=True)
    f_true.replace('3313901','s',inplace=True)
    f_true.replace('3313906','s',inplace=True)
    f_true.replace('3313902','t',inplace=True)
    f_true.replace('3313905','t',inplace=True)
    f_true.replace('3434801','u',inplace=True)
    f_true.replace('3434806','u',inplace=True)
    f_true.replace('3434803','v',inplace=True)
    f_true.replace('3434807','v',inplace=True)
    f_true.replace('3477801','w',inplace=True)
    f_true.replace('3477803','w',inplace=True)
    f_true.replace('3597906','x',inplace=True)
    f_true.replace('3597907','x',inplace=True)
    f_true.replace('3597911','x',inplace=True)
    f_true.replace('3893501','y',inplace=True)
    f_true.replace('3893505','y',inplace=True)
    f_true.replace('3893507','y',inplace=True)
    f_true.replace('3893508','y',inplace=True)
    f_true.replace('3893502','z',inplace=True)
    f_true.replace('3893503','z',inplace=True)
    f_true.replace('3893509','z',inplace=True)
    f_true.replace('4155302','aa',inplace=True)
    f_true.replace('4155303','aa',inplace=True)
    f_true.replace('4155304','aa',inplace=True)
    f_true.replace('4155307','aa',inplace=True)
    f_true.replace('4198501','bb',inplace=True)
    f_true.replace('4198502','bb',inplace=True)
    f_true.replace('4198503','bb',inplace=True)
    f_true.replace('4198506','bb',inplace=True)
    f_true.replace('4198507','bb',inplace=True)  
    f_true.replace('4380101','cc',inplace=True)
    f_true.replace('4380102','cc',inplace=True)
    f_true.replace('4380103','cc',inplace=True)
    f_true.replace('4380104','dd',inplace=True)
    f_true.replace('4380105','dd',inplace=True)

    #--------------------------------------------
    f_true.replace('4380106','ee',inplace=True)
    f_true.replace('4380109','ee',inplace=True)
    f_true.replace('4530702','ff',inplace=True)
    f_true.replace('4530704','ff',inplace=True)

    f_true.replace('4636102','gg',inplace=True) 
    f_true.replace('4636101','gg',inplace=True) 
    f_true.replace('4636107','gg',inplace=True) 
    f_true.replace('4636108','gg',inplace=True) 
    f_true.replace('4636103','hh',inplace=True) 
    f_true.replace('4636104','hh',inplace=True) 
    f_true.replace('4636110','hh',inplace=True) 
    f_true.replace('4636111','hh',inplace=True) 
    f_true.replace('4636112','hh',inplace=True) 
    #---------------

    f_true.replace('4640501','ii',inplace=True) 
    f_true.replace('4640502','ii',inplace=True) 
    f_true.replace('4640507','ii',inplace=True)
    f_true.replace('4640503','jj',inplace=True) 
    f_true.replace('4640508','jj',inplace=True) 
    f_true.replace('4640509','jj',inplace=True) 
    #------------------
    f_true.replace('4711401','kk',inplace=True) 
    f_true.replace('4711403','kk',inplace=True) 
    f_true.replace('4711405','kk',inplace=True)

    f_true.replace('4711404','ll',inplace=True) 
    f_true.replace('4711406','ll',inplace=True) 

    #---------------------------------
    f_true.replace('4753401','mm',inplace=True) 
    f_true.replace('4753404','mm',inplace=True) 
    f_true.replace('4753406','mm',inplace=True)
    f_true.replace('4753403','nn',inplace=True) 
    f_true.replace('4753407','nn',inplace=True)
    f_true.replace('4753405','oo',inplace=True) 
    f_true.replace('4753408','oo',inplace=True)



    #ADJ  合并后意思 2.75， 141个例子， 4个单词

    f_adj_answer.replace('hot%3:00:01::','a',inplace=True)
    f_adj_answer.replace('hot%3:00:02::','a',inplace=True)
    f_adj_answer.replace('hot%5:00:00:active:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:charged:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:fast:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:fresh:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:good:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:illegal:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:lucky:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:near:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:new:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:popular:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:pungent:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:radioactive:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:sexy:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:skilled:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:unpleasant:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:violent:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:wanted:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:warm:03','b',inplace=True)
    f_adj_answer.replace('hot%5:00:02:fast:01','b',inplace=True)
    f_adj_answer.replace('important%3:00:00::','c',inplace=True)
    f_adj_answer.replace('important%3:00:02::','c',inplace=True)
    f_adj_answer.replace('important%3:00:04::','c',inplace=True)
    f_adj_answer.replace('important%5:00:00:immodest:02','d',inplace=True)
    f_adj_answer.replace('important%5:00:00:influential:00','d',inplace=True)
    f_adj_answer.replace('solid%3:00:01::','e',inplace=True)
    f_adj_answer.replace('solid%3:00:02::','e',inplace=True)
    f_adj_answer.replace('solid%5:00:00:homogeneous:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:honorable:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:opaque:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:plain:02','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:sound:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:unbroken:02','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:undiversified:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:wholesome:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:hard:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:good:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:frozen:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:cubic:00','f',inplace=True)

    #
    #answer
    #-----------------

    f_adj_true.replace('hot%3:00:01::','a',inplace=True)
    f_adj_true.replace('hot%3:00:02::','a',inplace=True)
    f_adj_true.replace('hot%5:00:00:active:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:charged:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:fast:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:fresh:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:good:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:illegal:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:lucky:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:near:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:new:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:popular:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:pungent:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:radioactive:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:sexy:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:skilled:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:unpleasant:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:violent:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:wanted:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:warm:03','b',inplace=True)
    f_adj_true.replace('hot%5:00:02:fast:01','b',inplace=True)
    f_adj_true.replace('important%3:00:00::','c',inplace=True)
    f_adj_true.replace('important%3:00:02::','c',inplace=True)
    f_adj_true.replace('important%3:00:04::','c',inplace=True)
    f_adj_true.replace('important%5:00:00:immodest:02','d',inplace=True)
    f_adj_true.replace('important%5:00:00:influential:00','d',inplace=True)
    f_adj_true.replace('solid%3:00:01::','e',inplace=True)
    f_adj_true.replace('solid%3:00:02::','e',inplace=True)
    f_adj_true.replace('solid%5:00:00:homogeneous:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:honorable:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:opaque:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:plain:02','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:sound:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:unbroken:02','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:undiversified:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:wholesome:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:hard:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:good:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:frozen:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:cubic:00','f',inplace=True)
